In [1]:
%%sh

cd raw_data/

ls

pwd

sh: 2: cd: can't cd to raw_data/


News Article Analysis.ipynb
News Bias Image Analysis.ipynb
/mnt/raid1/MSCs/AY2425/skaul/diss_crawler/notebooks


In [8]:
data_path="../newscrawler/data"




news_path={"cnn": f"{data_path}/cnn_articles_20250601.json",
"ht": f"{data_path}/hindustan_times_articles_20250601.json",
"ind": f"{data_path}/india_articles_20250528.json",
"nbc": f"{data_path}/nbcnews_articles_20250528.json",
"news18": f"{data_path}/news18_articles_20250601.json",
"nweek": f"{data_path}/newsweek_articles_20250601.json",
"nypost": f"{data_path}/nypost_articles_20250601.json",
"bbc": f"{data_path}/bbcnews_articles_20250513.json",
"usa": f"{data_path}/usatoday_articles_20250531.json",
"ind_exp": f"{data_path}/indian_express_articles_20250601.json"
}

In [9]:
import json

def read_top_articles(file_path, num_articles=5):
  """
  Reads the top N articles from a JSON file.

  Args:
    file_path (str): The path to the JSON file.
    num_articles (int): The number of top articles to read.

  Returns:
    list: A list of dictionaries representing the top articles.
  """
  try:
    with open(file_path, 'r') as f:
      data = json.load(f)
      # Assuming the JSON structure contains a list of articles
      if isinstance(data, list):
        return data[:num_articles]
      else:
        print(f"Warning: Expected a list of articles in {file_path}")
        return []
  except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
    return []
  except json.JSONDecodeError:
    print(f"Error: Could not decode JSON from {file_path}")
    return []

for src,path in news_path.items():
    # Example usage with one of the file paths defined previously:
    print(f"Reading {src}")
    file_to_read = path
    top_articles = read_top_articles(file_to_read, 5)

    # Print the titles of the top articles (assuming 'title' key exists)
    if top_articles:
      print(f"Top 5 articles from {file_to_read}:")
      for article in top_articles:
        if 'title' in article:
          print(f"- {article['images']}")
        else:
          print("- Article does not have a 'title' key")
    else:
      print(f"No articles found or an error occurred for {file_to_read}")

Reading cnn
Top 5 articles from ../newscrawler/data/cnn_articles_20250601.json:
- ['https://media.cnn.com/api/v1/images/stellar/prod/gettyimages-2210108622.jpg?c=16x9&q=h_833,w_1480,c_fill']
- ['https://media.cnn.com/api/v1/images/stellar/prod/2025-05-01t165517z-1251819659-rc249ea0ovna-rtrmadp-3-may-day-usa.jpg?c=16x9&q=w_1280,c_fill', 'https://media.cnn.com/api/v1/images/stellar/prod/2025-05-01t165517z-1251819659-rc249ea0ovna-rtrmadp-3-may-day-usa.jpg?c=16x9&q=w_250,c_fill', 'https://media.cnn.com/api/v1/images/stellar/prod/actor-the-wire-tornado.jpg?c=16x9&q=w_250,c_fill', 'https://media.cnn.com/api/v1/images/stellar/prod/250528-musk-trump-bill-16x9.jpg?c=16x9&q=w_250,c_fill', 'https://media.cnn.com/api/v1/images/stellar/prod/gettyimages-2216096432.jpg?c=16x9&q=w_250,c_fill', 'https://media.cnn.com/api/v1/images/stellar/prod/punisher.jpg?c=16x9&q=w_250,c_fill', 'https://media.cnn.com/api/v1/images/stellar/prod/gettyimages-2216765160.jpg?c=16x9&q=w_250,c_fill', 'https://media.cnn.com/

In [12]:
import os, json, re
import pandas as pd

# ── POINT TO YOUR RAW DATA FOLDER ─────────────────────────────────────────
data_dir = "../newscrawler/data"       # or "/content/raw_data" in Colab

# ── LOAD & COMPUTE PER‐ARTICLE METRICS ────────────────────────────────────
records = []
for fn in os.listdir(data_dir):
    if not fn.endswith(".json"): continue
    src = fn.split("_articles")[0]
    path = os.path.join(data_dir, fn)
    with open(path, 'r') as f:
        arts = json.load(f)
    for i, art in enumerate(arts, start=1):
        # combine title/body
        txt = art.get("title","") + " "
        body = art.get("text","")
        txt += " ".join(body) if isinstance(body, list) else body

        # basic counts
        words = re.findall(r"\w+", txt)
        n_words = len(words)
        n_images = len(art.get("images", []))

        # framing terms
        killed = len(re.findall(r"\bkilled\b", txt, flags=re.I))
        died   = len(re.findall(r"\bdied\b",   txt, flags=re.I))

        # side mentions
        israel     = len(re.findall(r"\bIsrael\b",     txt, flags=re.I))
        palestine  = len(re.findall(r"\bPalestine\b",  txt, flags=re.I))

        records.append({
            "source": src,
            "article_idx": i,
            "n_words":    n_words,
            "n_images":   n_images,
            "killed":     killed,
            "died":       died,
            "mentions_Israel":    israel,
            "mentions_Palestine": palestine
        })

df = pd.DataFrame.from_records(records)

# ── AGGREGATE PER SOURCE ──────────────────────────────────────────────────
summary = df.groupby("source").agg(
    total_articles        = ("article_idx", "size"),
    avg_words             = ("n_words",    "mean"),
    avg_images            = ("n_images",   "mean"),
    total_killed_mentions = ("killed",     "sum"),
    total_died_mentions   = ("died",       "sum"),
    total_Israel_mentions = ("mentions_Israel",    "sum"),
    total_Palestine_mentions = ("mentions_Palestine", "sum"),
).reset_index()

# display as markdown table
print(summary.to_markdown(index=False))


| source          |   total_articles |   avg_words |   avg_images |   total_killed_mentions |   total_died_mentions |   total_Israel_mentions |   total_Palestine_mentions |
|:----------------|-----------------:|------------:|-------------:|------------------------:|----------------------:|------------------------:|---------------------------:|
| bbcnews         |             4848 |     700.366 |     3.25701  |                    6580 |                   872 |                   21680 |                       1019 |
| cnn             |             4190 |    1114.92  |     6.87112  |                    4759 |                   834 |                   26598 |                        848 |
| hindustan_times |             6619 |     586.012 |     0.957546 |                    7584 |                   798 |                   36067 |                       1754 |
| india           |             1326 |     860.347 |     0.999246 |                     930 |                   129 |                  

In [13]:
import os
import json
import re
import pandas as pd

# ── POINT THIS AT YOUR RAW JSONS ─────────────────────────────────────────────
# e.g. data_dir = "../raw_data" or "/content/raw_data"
data_dir = "../newscrawler/data"

# ── LOAD & COMPUTE PRELIMINARY METRICS ───────────────────────────────────────
records = []
for fn in os.listdir(data_dir):
    if not fn.endswith(".json"):
        continue
    source = fn.split("_articles")[0]
    path = os.path.join(data_dir, fn)
    with open(path, "r") as f:
        articles = json.load(f)

    for i, art in enumerate(articles, start=1):
        title = art.get("title", "").strip()
        body  = art.get("text", "")
        if isinstance(body, list):
            body_text = " ".join(body)
        else:
            body_text = (body or "")
        text = f"{title} {body_text}"

        #Headline length in words
        headline_len   = len(re.findall(r"\w+", title))

        #Vocabulary size (unique tokens in the whole article)
        tokens         = re.findall(r"\w+", text.lower())
        vocab_size     = len(set(tokens))

        #Quote density (# of quoted spans)
        quote_count    = len(re.findall(r'“[^”]+”|\"[^\"]+\"', text))

        #Passive-voice proxy (“was Xed”)
        passive_count  = len(re.findall(r"\bwas\s+\w+ed\b", text, flags=re.I))

        #Modal verbs (should/must/ought)
        modal_count    = len(re.findall(r"\b(should|must|ought)\b", text, flags=re.I))

        #External links count
        links_count    = len(re.findall(r"https?://", text))

        records.append({
            "source": source,
            "article_idx": i,
            "headline_len": headline_len,
            "vocab_size": vocab_size,
            "quote_count": quote_count,
            "passive_count": passive_count,
            "modal_count": modal_count,
            "external_links": links_count
        })

df = pd.DataFrame(records)


agg = df.groupby("source").agg(
    total_articles      = ("article_idx",    "size"),
    avg_headline_len    = ("headline_len",   "mean"),
    avg_vocab_size      = ("vocab_size",     "mean"),
    avg_quote_count     = ("quote_count",    "mean"),
    avg_passive_count   = ("passive_count",  "mean"),
    avg_modal_count     = ("modal_count",    "mean"),
    avg_external_links  = ("external_links", "mean"),
).reset_index()


print(agg.to_markdown(index=False))


| source          |   total_articles |   avg_headline_len |   avg_vocab_size |   avg_quote_count |   avg_passive_count |   avg_modal_count |   avg_external_links |
|:----------------|-----------------:|-------------------:|-----------------:|------------------:|--------------------:|------------------:|---------------------:|
| bbcnews         |             4848 |             0.3125 |          324.623 |          10.7632  |            1.42347  |          0.685025 |          0           |
| cnn             |             4190 |            14.3819 |          469.284 |          14.7704  |            1.61122  |          0.931026 |          0           |
| hindustan_times |             6619 |            12.872  |          279.671 |           6.29612 |            1.12736  |          0.525608 |          0.0256836   |
| india           |             1326 |            16.3718 |          353.645 |          30.1742  |            0.894419 |          0.453243 |          0.147059    |
| indian_express

In [19]:
print(agg[["source","avg_headline_len","avg_headline_len"]].to_csv(index=False))

source,avg_headline_len,avg_headline_len
bbcnews,0.3125,0.3125
cnn,14.381861575178998,14.381861575178998
hindustan_times,12.872035050611874,12.872035050611874
india,16.371794871794872,16.371794871794872
indian_express,12.17517087903354,12.17517087903354
nbcnews,13.243414634146342,13.243414634146342
news18,13.71400683260127,13.71400683260127
newsweek,9.99190023142196,9.99190023142196
nypost,15.677741761210157,15.677741761210157
usatoday,12.654998423210344,12.654998423210344



In [26]:
import json
import os

# Path to the JSON file with all articles
json_path = os.path.join(data_dir, "nypost_articles_20250601.json")

# Load articles
with open(json_path, "r") as f:
    articles = json.load(f)

# Sort articles by number of images (descending)
articles_sorted = sorted(
    articles,
    key=lambda x: len(x.get("images", [])),
    reverse=True
)

# Print top 15 articles with most images
print("Top 15 articles with most images:\n")
for idx, article in enumerate(articles_sorted[:15], 1):
    title = article.get("title", "No Title")
    n_images = len(article.get("images", []))
    url = article.get("url", "")
    print(f"{idx}. {title}\n   Images: {n_images}\n   URL: {url}\n")

Top 15 articles with most images:

1. These 97 people are still being held by Hamas a year after Oct. 7 attacks
   Images: 96
   URL: https://nypost.com/2024/10/07/world-news/faces-of-oct-7-hostages-97-people-still-held-by-hamas-in-gaza/

2. The world reacts to Donald Trump’s historic 2024 presidential election win in newspaper front pages
   Images: 34
   URL: https://nypost.com/2024/11/06/us-news/the-world-reacts-to-donald-trumps-historic-2024-presidential-election-win-in-newspaper-front-pages/

3. Hurricane Milton in photos
   Images: 31
   URL: https://nypost.com/us-news/hurricane-milton-hits-florida-photos/

4. Campus comrades: Anti-Israel tent cities and mass protests spring up at elite — and pricey — colleges across US
   Images: 28
   URL: https://nypost.com/2024/04/22/us-news/college-anti-israel-solidarity-protests-spring-up-across-us/

5. Hunter Biden’s ‘ashamed’ sister-in-law, ex-lover Hallie says ‘I was just panicking’ when she found first son’s gun, testifies he introduced

In [27]:
# Path to the JSON file with all BBC articles
bbc_path = os.path.join(data_dir, "bbcnews_articles_20250513.json")

# Load BBC articles
with open(bbc_path, "r") as f:
    bbc_articles = json.load(f)

# Sort articles by number of images (descending)
bbc_sorted = sorted(
    bbc_articles,
    key=lambda x: len(x.get("images", [])),
    reverse=True
)

# Print top 15 BBC articles with most images
print("Top 15 BBC articles with most images:\n")
for idx, article in enumerate(bbc_sorted[:15], 1):
    title = article.get("title", "No Title")
    n_images = len(article.get("images", []))
    url = article.get("url", "")
    print(f"{idx}. {title}\n   Images: {n_images}\n   URL: {url}\n")


Top 15 BBC articles with most images:

1. 
   Images: 58
   URL: https://www.bbc.com/news/articles/c789l8p8j7lo

2. The Netherlands, Israel and Switzerland among countries heading to the Eurovision final
   Images: 35
   URL: https://www.bbc.com/news/live/cp4ged0xv4zt

3. 
   Images: 34
   URL: https://www.bbc.com/news/world-middle-east-67477240

4. 
   Images: 29
   URL: https://www.bbc.com/news/articles/cj4vpd1yrdvo

5. No room in shelters as thousands flee to Gaza border city
   Images: 28
   URL: https://www.bbc.com/news/live/world-middle-east-67633071

6. 
   Images: 27
   URL: https://www.bbc.com/news/world-asia-india-68823827

7. 
   Images: 25
   URL: https://www.bbc.com/news/articles/clylzx1xz2yo

8. 'We keep our bags packed in case we need to flee': Follow daily life in Gaza
   Images: 24
   URL: https://www.bbc.com/news/live/world-middle-east-68339508

9. Israel orders more Rafah evacuations and reports face-to-face battles with Hamas
   Images: 23
   URL: https://www.bbc.co

In [28]:
# Path to the JSON file with all CNN articles
cnn_path = os.path.join(data_dir, "cnn_articles_20250601.json")

# Load CNN articles
with open(cnn_path, "r") as f:
    cnn_articles = json.load(f)

# Sort articles by number of images (descending)
cnn_sorted = sorted(
    cnn_articles,
    key=lambda x: len(x.get("images", [])),
    reverse=True
)

# Print top 15 CNN articles with most images
print("Top 15 CNN articles with most images:\n")
for idx, article in enumerate(cnn_sorted[:15], 1):
    title = article.get("title", "No Title")
    n_images = len(article.get("images", []))
    url = article.get("url", "")
    print(f"{idx}. {title}\n   Images: {n_images}\n   URL: {url}\n")


Top 15 CNN articles with most images:

1. 2,000 children killed in Gaza, aid group says, as tempers flare at UN amid ceasefire calls
   Images: 370
   URL: https://www.cnn.com/2023/10/24/middleeast/israel-hamas-gaza-war-tuesday-intl-hnk/index.html

2. Israeli defense minister orders ‘complete siege’ of Gaza, as Hamas threatens hostages
   Images: 359
   URL: https://www.cnn.com/2023/10/09/middleeast/israel-gaza-hamas-fighting-monday-intl-hnk

3. Netanyahu says Israel is ‘at war’ after Hamas launches surprise air and ground attack from Gaza
   Images: 357
   URL: https://www.cnn.com/2023/10/07/middleeast/sirens-israel-rocket-attack-gaza-intl-hnk

4. Hamas releases two more hostages as Israel ramps up Gaza strikes
   Images: 357
   URL: https://www.cnn.com/2023/10/23/middleeast/israel-gaza-hamas-war-monday-intl-hnk

5. Hamas and Israeli troops clash in Gaza as airstrikes intensify
   Images: 356
   URL: https://www.cnn.com/2023/10/22/middleeast/israel-gaza-hamas-war-sunday-intl-hnk

6. ‘

In [23]:
%%markdown

EXAMPLES:

NyPOST:

BBC:
CNN:



EXAMPLES:

NyPOST:

BBC:
CNN:
